## For Local master node

In [ ]:
%%local
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
spark.sparkContext.getConf().getAll()
spark.sparkContext.version
!which pyspark

In [ ]:
%%local
from pyspark.sql import SparkSession
from sagemaker_pyspark import classpath_jars # for S3 access
#spark=SparkSession.builder.config("spark.driver.extraClassPath", ":".join(classpath_jars())).config('spark.jars.packages','org.apache.spark:spark-avro_2.11:2.4.0').getOrCreate()
spark=SparkSession.builder.config('spark.jars.packages','org.apache.spark:spark-avro_2.11:2.4.0').config("spark.driver.extraClassPath", ":".join(classpath_jars())).config('spark.jars','file:///home/ec2-user/.ivy2/jars/org.apache.spark_spark-tags_2.11-2.4.0.jar,file:///home/ec2-user/.ivy2/jars/org.apache.spark_spark-avro_2.11-2.4.0.jar,file:///home/ec2-user/.ivy2/jars/org.spark-project.spark_unused-1.0.0.jar').getOrCreate()
spark=SparkSession.builder.config('spark.jars.packages','org.apache.spark:spark-avro_2.11:2.4.0').getOrCreate()
# spark.conf.set('spark.jars.packages','org.apache.spark:spark-avro_2.11:2.4.0')
#dF = spark.read.format("avro").load("/home/ec2-user/0.avro").show()
dF = spark.read.format("avro").load("s3a://sfdmdb/SF_CASE_VW/ymd=2018*/0.avro").count()
#spark.sparkContext.getConf().getAll()
#terminfo = spark.read.parquet("s3a://qaessential/jup2_stats.terminal_info/ymd=20181101").show()

In [ ]:
%%local
dF

## For Remote clusters

In [ ]:
%%info

In [ ]:
%%configure -f
{ "conf": {"spark.jars.packages": "org.apache.spark:spark-avro_2.11:2.4.0","spark.jars":"/opt/flint.jar"} }

In [1]:
#spark.conf.set('spark.jars.packages','org.apache.spark:spark-avro_2.11:2.4.0')
spark.read.format("avro").load("s3a://sfdmdb/SF_CASE_VW/ymd=20181101/0.avro").show()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1551802503247_0014,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
+----------+--------------------+--------------------+------+-------------+-------------+-----------------+------------------------+--------------------+----------------+--------------------+--------------------+--------------+--------------+-------------------+---------------------+--------------------+-----------------+-------------+---------------+------------------+--------------------+----------------+--------------------------+--------------------+-------------------+------------------------------+-------------------+----------+-----------------------+------------------+
|CASENUMBER|         CREATEDDATE|          CLOSEDDATE|REASON|         TYPE| CASE_TYPE__C|  CASE_SUBTYPE__C|CUSTOMER_REQUEST_TYPE__C|           ACTION__C|ACTIVITY_TYPE__C|     ACTUAL_ISSUE__C|              STATUS|ACCOUNT_SAN__C|CASE_STATUS__C|CASE_RECORD_TYPE__C|PARENT_CUSTOMER_ID__C|  CONTRACT_STATUS__C|CUSTOMER_VALUE__C|CHURN_RISK__C|HSVC_BEAM_ID__C|HSVC_GATEWAY_ID__C|HSVC_SATE

In [5]:
from pyspark.sql.functions import to_timestamp,from_unixtime
from ts.flint import FlintContext
flintContext = FlintContext(sqlContext)
df = spark.read.format('avro').load("s3a://vsat-lmtd/jup2_stats/vsat_stcd/year=2019/month=2/day=1/*").select(to_timestamp(from_unixtime("collection_start_time", "yyyy-MM-dd HH")).alias('time'),'device_id','statecode','statecodeduration')
df.printSchema
vsatlmtd= flintContext.read.dataframe(df)
vsatlmtd.show()

"cannot resolve 'from_utc_timestamp(`collection_start_time`, 'UTC')' due to data type mismatch: argument 1 requires timestamp type, however, '`collection_start_time`' is of bigint type.;;\n'Project [collection_start_time#255L, device_id#253, statecode#257, statecodeduration#258L, from_utc_timestamp(collection_start_time#255L, UTC) AS time#279]\n+- Project [collection_start_time#255L, device_id#253, statecode#257, statecodeduration#258L]\n   +- Relation[ingest_batch_id#247L,ingest_time#248L,header_hash#249,schema_version#250,unknown_schema#251,gateway_id#252,device_id#253,esn#254L,collection_start_time#255L,collection_end_time#256L,statecode#257,statecodeduration#258L,statecodelastoccurrencestart#259,statecodelastoccurrenceend#260] avro\n"
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 1989, in withColumn
    return DataFrame(self._jdf.withColumn(colName, col._jc), self.sql_ctx)
  File "/usr/lib/spark/python/lib/py4j-0.10

In [ ]:
#spark.hadoop.parquet.enable.summary-metadata false
#spark.sql.parquet.mergeSchema false
#spark.sql.parquet.filterPushdown true
#spark.sql.hive.metastorePartitionPruning true
terminfo = spark.read.parquet("s3a://qaessential/jup2_stats.terminal_info/ymd=20181101").select('year','month','day','device_id','hw_type','absolute_beam_id')
vsatlmtd = spark.read.parquet("s3a://qaessential/jup2_stats.vsat_stcd/ymd=20181101").select('year','month','day','device_id','statecode','statecodeduration')
terminfo.createOrReplaceTempView("terminfo")
vsatlmtd.createOrReplaceTempView("vsatlmtd")
vsatlmtd_e=vsatlmtd.join(terminfo,['year','month','day','device_id'])
vsatlmtd_e.explain()
vsatlmtd_e.show()

In [ ]:
%%info

In [ ]:
%%sql -o result
select * from terminfo join vsatlmtd on terminfo.year=vsatlmtd.year and terminfo.month=vsatlmtd.month and terminfo.day=vsatlmtd.day and terminfo.device_id=vsatlmtd.device_id

In [ ]:
%%sh
git add .
git commit -m 'ingest' -a
git push origin master